#### Импорт необходимых библиотек

In [ ]:
import json
import os
import jiwer as jiwer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from vosk import Model, KaldiRecognizer
import wave
import numpy as np
from pydub import AudioSegment
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import process
from transformers import BertTokenizer, BertModel
import optuna
from gensim.models import Word2Vec
import librosa

In [ ]:
# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    'hr_bot_clear',
    'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    'hr_bot_clear.json',
    'hr_bot_noise.json',
    'hr_bot_synt.json'
]

VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')

# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"  # Путь к скачанной модели Vosk

In [ ]:
# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"
model = Model(MODEL_PATH)
print("Модель Vosk загружена успешно.")

In [ ]:
# Метки команд
_label = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

#### Объявление классов

In [ ]:
# Датасет для классификации текста
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [ ]:
# Нейронная сеть для классификации текста
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

#### Объявление функций

In [ ]:
def prepare_word2vec(texts):
    tokenized_texts = [text.split() for text in texts]
    model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)
    return model

In [ ]:
def encode_with_bert(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # Усреднение по всем токенам

In [ ]:
# Аугментация аудио
def augment_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)

    # Добавление белого шума
    noise = np.random.randn(len(audio))
    augmented_audio = audio + 0.005 * noise

    # Изменение скорости
    augmented_audio_speed = librosa.effects.time_stretch(audio, rate=1.2)

    # Save augmented audio
    augmented_file_paths = []
    for augmented_audio_variant in [augmented_audio, augmented_audio_speed]:
        augmented_file_path = file_path.replace('.wav', f'_{augmented_audio_variant}.wav')
        librosa.output.write_wav(augmented_file_path, augmented_audio_variant, sr)
        augmented_file_paths.append(augmented_file_path)

    return augmented_file_paths

In [ ]:
def correct_transcription(transcribed_text):
    # Используем fuzzywuzzy для нахождения ближайшей команды
    corrected_text, score = process.extractOne(transcribed_text, label.values())
    if score >= 80:  # Порог для принятия исправления
        return corrected_text
    return transcribed_text  # Если нет похожей команды, возвращаем оригинал

In [ ]:
# Преобразование MP3 в WAV
def convert_mp3_to_wav(mp3_filepath):
    try:
        # Убедимся, что pydub правильно обрабатывает MP3
        if not mp3_filepath.endswith('.mp3'):
            print(f"Файл {mp3_filepath} не является MP3.")
            return None

        wav_filepath = mp3_filepath.replace('.mp3', '.wav')
        audio = AudioSegment.from_file(mp3_filepath)  # Используем from_file для универсальности
        audio.export(wav_filepath, format='wav')
        print(f"Файл {mp3_filepath} успешно конвертирован в WAV: {wav_filepath}")
        return wav_filepath
    except Exception as e:
        print(f"Ошибка при конвертации {mp3_filepath} в WAV: {e}")
        return None

In [ ]:
# Функция для обработки аудиофайла
def transcribe_audio(audio_file, dir):
    wf = wave.open(f"{dir}/{audio_file}", "rb")
    rec = KaldiRecognizer(model, wf.getframerate())

    result_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            result_text += result.get("text", "") + " "

    final_result = json.loads(rec.FinalResult())
    result_text += final_result.get("text", "")
    print(f"Распознанный текст для {audio_file}: {result_text}")
    return result_text.strip()

In [ ]:
# Функция классификации текста
def classify_text(text, classifier, tokenizer):
    # Преобразование текста в вектор
    text_vector = tokenizer.transform([text]).toarray()
    text_tensor = torch.tensor(text_vector, dtype=torch.float32)

    # Классификация текста
    with torch.no_grad():
        outputs = classifier(text_tensor)

    _, predicted_class = torch.max(outputs, 1)

    return predicted_class.item()

In [ ]:
def process_audio_files(file_list, classifier, tokenizer, result_dir):
    results = []

    for file_info in file_list:
        audio_file = file_info['file']
        audio_id = file_info['id']
        print(f"Обработка файла: {audio_file}")

        # Транскрибируем аудиофайл
        transcribed_text = transcribe_audio(audio_file)
        if transcribed_text is None:
            print(f"Ошибка транскрипции для {audio_file}. Пропуск файла.")
            continue  # Пропускаем файл в случае ошибки

        # Коррекция текста
        corrected_text = correct_transcription(transcribed_text)
        print(f"Исправленный текст для {audio_file}: {corrected_text}")

        # Классификация текста
        predicted_class = classify_text(corrected_text, classifier, tokenizer)

        # Собираем атрибуты
        attributes = {
            "length": len(corrected_text.split()),
            "contains_numbers": any(char.isdigit() for char in corrected_text)
        }

        result = {
            "file_name": os.path.basename(audio_file),
            "file_id": audio_id,
            "transcription": corrected_text,
            "category": predicted_class,
            "attributes": attributes
        }

        results.append(result)
        print(f"Готово: {audio_file}")

    return results

#### Подготовка настроек модели, создание модели

In [ ]:
# Пример классификатора и токенизатора
input_dim = 2  # Зависит от метода векторизации текста
hidden_dim = 55
output_dim = len(_label)  # Количество классов
classifier = TextClassifier(input_dim, hidden_dim, output_dim)

#### Подготовка данных для обучения

In [ ]:
# Функция загрузки аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [ ]:
def load_dataset():
    audio_files = []
    texts = []
    labels = []

    for annotation_file, data_dir in zip(ANNOTATION_FILES, DATA_DIR_FILES):
        annotation_path = os.path.join(ANNOTATION_DIR, annotation_file)
        print(f"Загрузка аннотаций из: {annotation_path}")
        training_annotations = load_annotations(annotation_path)

        for annotation in training_annotations:
            audio_filepath = os.path.join(DATA_DIR, data_dir, annotation['audio_filepath'])
            # Проверка существования файла перед конвертацией
            if os.path.exists(audio_filepath):
                if audio_filepath.endswith('.mp3'):
                    audio_filepath = convert_mp3_to_wav(audio_filepath)

                augmented_files = augment_audio(audio_filepath)  # Аугментация аудио
                audio_files.extend(augmented_files)

            else:
                print(f"Файл {audio_filepath} не найден.")
                continue  # Пропустить, если файл не найден


            text = annotation['text']
            label = annotation['label']
            texts.extend([text] * len(augmented_files))  # Повторяем текст для каждого аугментированного файла
            labels.extend([label] * len(augmented_files))

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels

print("Начало загрузки данных...")
audio_files, texts, labels = load_dataset()

In [ ]:
# Использование BERT для контекстуальной векторизации
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Подготовка данных для Word2Vec
word2vec_model = prepare_word2vec(texts)
word2vec_vectors = np.array([word2vec_model.wv[text.split()].mean(axis=0) for text in texts])

# Подготовка TF-IDF векторов
tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Максимальное количество признаков
tfidf_vectors = tfidf_vectorizer.fit_transform(texts).toarray()

# Пример использования BERT
bert_vectors = np.array([encode_with_bert(text).detach().numpy() for text in texts])

In [ ]:
print("Разделение данных на тренировочные и тестовые...")
X_train, X_val, y_train, y_val = train_test_split(tfidf_vectors, labels, test_size=0.2, random_state=42)

In [ ]:
# Create data loaders
train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Initialize classifier
input_dim = X_train.shape[1]
hidden_dim = 64  # Can be tuned
output_dim = len(set(labels))  # Number of classes
classifier = TextClassifier(input_dim, hidden_dim, output_dim)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

#### Функция обучения модели (+ расчет метрик каждые 5 эпох)

In [ ]:
# Training loop
num_epochs = 10  # Can be tuned
for epoch in range(num_epochs):
    classifier.train()
    for texts, labels in train_loader:
        optimizer.zero_grad()
        outputs = classifier(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    classifier.eval()
    val_outputs = []
    val_labels = []
    with torch.no_grad():
        for texts, labels in val_loader:
            outputs = classifier(texts)
            val_outputs.append(outputs)
            val_labels.append(labels)

    val_outputs = torch.cat(val_outputs)
    val_labels = torch.cat(val_labels)

    val_predictions = torch.argmax(val_outputs, dim=1).numpy()
    val_f1_score = f1_score(y_val, val_predictions, average='weighted')
    val_precision = precision_score(y_val, val_predictions, average='weighted')
    val_recall = recall_score(y_val, val_predictions, average='weighted')

    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Validation F1 Score: {val_f1_score:.4f} - "
          f"Validation Precision: {val_precision:.4f} - "
          f"Validation Recall: {val_recall:.4f}")

# Save the trained model
torch.save(classifier.state_dict(), '../model/text_classifier.pth')
print("Model saved.")

#### Метрики производительности

In [ ]:
# Validate audio files
audio_files_to_validate = [audio_file for audio_file in audio_files if audio_file.endswith('.wav')]
print(f"Starting audio validation for {len(audio_files_to_validate)} files...")

validation_results = process_audio_files(audio_files_to_validate, classifier)

In [ ]:
# Save results to JSON
results_file_path = '../data/results/validation_results.json'
with open(results_file_path, 'w', encoding='utf-8') as results_file:
    json.dump(validation_results, results_file, ensure_ascii=False, indent=4)

print(f"Validation results saved to {results_file_path}.")

In [ ]:
#### Метрики производительности

In [ ]:
# Вычисление WER
def calculate_wer(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    S = sum(1 for r, h in zip(reference_words, hypothesis_words) if r != h)
    D = len(reference_words) - len(hypothesis_words) if len(reference_words) > len(hypothesis_words) else 0
    I = len(hypothesis_words) - len(reference_words) if len(hypothesis_words) > len(reference_words) else 0
    N = len(reference_words)

    return (S + D + I) / N if N > 0 else 0

# Вычисление Mq
def calculate_mq(wer, f1_weighted):
    WERnorm = wer
    return 0.25 * (1 - WERnorm) + 0.75 * f1_weighted

In [ ]:
# Оптимизация гиперпараметров с помощью Optuna
def objective(trial):
    hidden_dim = trial.suggest_int('hidden_dim', 32, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 4, 16)

    # Загрузка и подготовка данных
    audio_files, texts, labels = load_dataset()
    train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

    # Генерация векторов BERT
    bert_vectors = np.array([encode_with_bert(text).detach().numpy() for text in train_texts])

    # Создание датасета и загрузчика
    train_dataset = TextDataset(bert_vectors, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Обучение модели
    classifier = TextClassifier(input_dim=768, hidden_dim=hidden_dim, output_dim=len(set(labels)))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

    for epoch in range(3):  # Можно увеличить количество эпох
        for texts_batch, labels_batch in train_loader:
            texts_batch = texts_batch.float()
            optimizer.zero_grad()
            outputs = classifier(texts_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

    # Оценка модели на тестовом наборе
    test_bert_vectors = np.array([encode_with_bert(text).detach().numpy() for text in test_texts])
    test_dataset = TextDataset(test_bert_vectors, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for texts_batch, labels_batch in test_loader:
            texts_batch = texts_batch.float()
            outputs = classifier(texts_batch)
            _, predicted = torch.max(outputs, 1)

            all_predictions.extend(predicted.numpy())
            all_labels.extend(labels_batch.numpy())

    # Вычисление метрик
    f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')

    # Расчет WER и Mq
    wer = calculate_wer(" ".join(test_texts), " ".join([tokenizer.decode(pred) for pred in all_predictions]))
    mq = calculate_mq(wer, f1_weighted)

    print(f'F1-Weighted: {f1_weighted}, Precision: {precision}, Recall: {recall}, WER: {wer}, Mq: {mq}')

    return f1_weighted

# Запуск оптимизации гиперпараметров
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print("Лучшие гиперпараметры: ", study.best_params)

#### Проверка модели на тестовых данных

In [ ]:
def process_validation_files(val_dir, classifier, tokenizer):
    """Обрабатывает валидационные файлы и сохраняет результаты."""
    annotations = load_annotations(ANNOTATION_VAL_FILE)
    print(f"Загружено аннотаций: {len(annotations)}")

    result_files = []
    for annotation in annotations:
        audio_filepath = os.path.join(val_dir, annotation['audio_filepath'])
        if os.path.exists(audio_filepath):
            result_files.append({"file": audio_filepath, "id": annotation['id']})
        else:
            print(f"Файл {audio_filepath} не найден.")

    if result_files:
        # Обрабатываем аудиофайлы
        transcription_results = process_audio_files(result_files, classifier, tokenizer, val_dir)

        # Сохранение результатов в JSON
        output_file = "transcriptions_validation.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(transcription_results, f, ensure_ascii=False, indent=4)

        print(f"Результаты транскрипции сохранены в {output_file}")
    else:
        print("Нет доступных аудиофайлов для обработки.")

# Вызов функции для обработки валидационных файлов
print("Обработка валидационных файлов...")
process_validation_files(VAL_DIR, classifier, tokenizer)